# Import train dataset

In [101]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

dataset = pd.read_csv('dota2Train.csv', header=None).as_matrix()

In [100]:
def filter_df(dataset):
    real_dataset = []
    for line in dataset:
        values = line[0:4].tolist()
        for i in range(4, len(line)):
            if line[i] == 1:
                values.append(i - 3)
            elif line[i] == -1:
                values.append((i - 3 )* -1)
        values = values[0:4] + sorted(values[4:])
        for i in range(4, 9):
            values[i] *= -1
        #values = values[0:4] + sorted(values[4:9]) + values[9:]
        real_dataset.append(values)
    return pd.DataFrame(real_dataset)

In [102]:
real_dataset = filter_df(dataset)
print(real_dataset[0:10])

x = real_dataset.iloc[0:, 4:]
y = real_dataset.iloc[:, 0]

   0    1   2   3    4   5   6   7   8   9   10  11  12   13
0  -1  223   2   2   88  74  38  22  18  10  14  25  28   32
1   1  152   2   2   98  93  35  21   6   4  14  26  27   71
2   1  131   2   2   93  72  46  20   6   4  22  25  32   59
3   1  154   2   2  104  47  42  23   7  17  35  54  62   95
4  -1  171   2   3   86  29  11   9   6  16  31  36  44   73
5   1  122   2   3  113  70  55  28  12   2  57  63  74  104
6   1  224   8   3   74  55  41  38  18  14  22  36  93   96
7  -1  227   8   3   98  93  74  62   2  14  32  34  72   75
8  -1  111   2   3   83  70  64  41   8   3  59  73  81  105
9  -1  151   2   2   87  84  44  32   8   4  30  46  59  106


# Correlation matrix

In [ ]:
import seaborn as sns

def table_corr(df):
    sns.heatmap(df.corr(), annot=True, cmap='summer')
    plt.show()
    
table_corr(real_dataset)

# Import test dataset

In [103]:
test_df = pd.read_csv('dota2Test.csv', header=None).as_matrix()
dataset_test = filter_df(test_df)

X_testd = dataset_test.iloc[0:, 4:]
y_testd = dataset_test.iloc[:, 0]

# KNN

In [104]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x, y)

pred = knn.predict(X_testd)

In [105]:
from sklearn.metrics import accuracy_score

acc_knn = accuracy_score(y_testd, pred)
print(acc_knn)

0.515251602875


# Naive Bayes

In [106]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(x, y)
pred_gnb = gnb.predict(X_testd)

In [107]:
acc_gnb = accuracy_score(y_testd, pred_gnb)
print(acc_gnb)

0.525840295318


# Decision Tree

In [108]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=10)
dt.fit(x, y)
pred_dt = dt.predict(X_testd)

In [109]:
acc_dt = accuracy_score(y_testd, pred_dt)
print(acc_dt)

0.531377501457


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=1000)
rfc.fit(x, y)
pred_rfc = rfc.predict(X_testd)

In [115]:
acc_rfc = accuracy_score(y_testd, pred_rfc)
print(acc_rfc)

0.533903244609
